In [1]:
import pysal as ps
import numpy as np
import json

In [2]:
ps.examples.explain('virginia')

{'description': 'Virginia counties shapefile',
 'explanation': ['  * virginia.shp: Shapefile',
  '  * virginia.shx: shapefile index',
  '  * virginia.dbf: attributes',
  '  * virginia.prj: shapefile projection',
  '  * virginia.json: geojson file'],
 'name': 'virginia'}

In [3]:
def geojson_handler(file_path):
    with open(file_path) as source:
        data = json.load(source)
    feature_type = data['features'][0]['geometry']['type']
    return data

In [4]:
data = geojson_handler(ps.examples.get_path('virginia.json'))

In [5]:
data.keys()

[u'crs', u'type', u'features']

In [6]:
f0 = data['features'][0]['geometry']['type']

In [7]:
f0

u'MultiPolygon'

In [8]:
f0

u'MultiPolygon'

In [53]:
class GeometryCollection(object):
    def __init__(self, geometries, bbox=None):
        self.geometries = geometries
        self.type = 'Abstract'
        self._cache = {}
        self.setbbox(bbox)
        
    def getbbox(self): 
        return self.__bbox
    def setbbox(self, value): 
        if value is None:
            value = self.build_bbox()
        self.__bbox = value
    bbox = property(getbbox, setbbox)  
    
    def build_bbox(self):
        x0 = y0 = np.Infinity
        x1 = y1 = -np.Infinity
        for geometry in self.geometries:
            print geometry
        

class PolygonCollection(GeometryCollection):
    def __init__(self, geometries, bbox=None):
        super(PolygonCollection, self).__init__(geometries, bbox)
        self.type = 'Polygon'
    
    def build_bbox(self):
        x0 = y0 = np.Infinity
        x1 = y1 = -np.Infinity
        for i, geometry in enumerate(self.geometries):
            nrings = len(geometry)
            for r, ring in enumerate(geometry):
                nc = len(ring)
                xs = [c[0] for c in ring]
                ys = [c[1] for c in ring]
                x0 = min(min(xs), x0)
                x1 = max(max(xs), x1)
                y0 = min(min(ys), y0)
                y1 = max(max(ys), y1)
                #print(i, nrings, nc)
        return([x0, x1, y0, y1])
        
    

class MultiPolygonCollection(GeometryCollection):
    def __init__(self, geometries, bbox=None):
        super(MultiPolygonCollection, self).__init__(geometries, bbox)
        self.type = 'MultiPolygon'
        
    def build_bbox(self):
        x0 = y0 = np.Infinity
        x1 = y1 = -np.Infinity
        for i, geometry in enumerate(self.geometries):
            npolys = len(geometry)
            nr = 0
            for j,poly in enumerate(geometry):
                nr = len(poly)
                
                for r,ring in enumerate(poly):
                    print(i, j, r, len(ring))
           
            print('\n')
            
    
class PointCollection(GeometryCollection):
    def __init__(self, geometries, bbox=None):
        super(PointCollection, self).__init__(geometries, bbox)
        self.type = 'Point'
        
    def build_bbox(self):
        x0 = y0 = np.Infinity
        x1 = y1 = -np.Infinity
        coords = np.array(list(self.geometries))
        maxc = coords.max(axis=0).tolist()
        minc = coords.min(axis=0).tolist()
        #print(coords)
        #print(minc, maxc)
        return([minc[0], minc[1], maxc[0], maxc[1]])
        
    
class LineCollection(GeometryCollection):
    def __init__(self, geometries, bbox=None):
        super(LineCollection, self).__init__(geometries, bbox)
        self.type = 'Point'
        
    def build_bbox(self):
        raise NotImplemented
    

In [54]:
dispatcher = {}
dispatcher[u'Polygon'] = PolygonCollection
dispatcher[u'MultiPolygon'] = MultiPolygonCollection
dispatcher[u'Point'] = PointCollection

def geojson_handler(file_path):
    with open(file_path) as source:
        data = json.load(source)
    feature_type = data['features'][0]['geometry']['type']
    geometries = (feature['geometry']['coordinates'] for feature in data['features'])
    collection = dispatcher[feature_type](geometries)
    return collection

#v = geojson_handler(ps.examples.get_path('virginia.json'))
b = geojson_handler(ps.examples.get_path('burkitt.geojson'))
c = geojson_handler(ps.examples.get_path('columbus.json'))


In [28]:
v.type

'MultiPolygon'

In [29]:
b.type

'Point'

In [13]:
b.geometries

<generator object <genexpr> at 0x7f765c3746e0>

In [30]:
c.type

'Polygon'

In [31]:
for g in c.geometries:
    print g

In [34]:
c.bbox

[5.87490701675415, 11.287420272827148, 10.788629531860352, 14.742449760437012]

In [55]:
b.bbox

[255.0, 247.0, 335.0, 399.0]